In [1]:
import mlrun

In [2]:
# load function from Github
xfn = mlrun.import_function('https://raw.githubusercontent.com/yjb-ds/functions/master/fileutils/function.yaml')

# configute it: mount on iguazio fabric, set as interactive (return stdout)
xfn.apply(mlrun.mount_v3io())
xfn.interactive = True

# create and run the task

images_path = '/User/mlrun/functions/images'

open_archive_task = mlrun.NewTask(
    'download',
    handler='open_archive', 
    params={'target_dir': images_path},
    inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'}
)

# run
run = xfn.run(open_archive_task)

/User/.pythonlibs/jupyter-1/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[mlrun] 2020-01-09 18:09:31,542 starting run download uid=643dd4c20c564bd199e5b77a250fd48c  -> 
[mlrun] 2020-01-09 18:09:31,544 warning!, Api url not set, trying to exec remote runtime locally
[mlrun] 2020-01-09 18:09:31,546 using in-cluster config.
[mlrun] 2020-01-09 18:09:31,560 Pod download-4dknd created
[mlrun] 2020-01-09 18:09:31,561 Job is running in the background, pod: download-4dknd


uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...0fd48c,0,,completed,download,kind=jobowner=admin,archive_url,target_dir=/User/mlrun/functions/images,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 643dd4c20c564bd199e5b77a250fd48c  , !mlrun logs 643dd4c20c564bd199e5b77a250fd48c 
[mlrun] 2020-01-09 18:09:31,615 run executed, status=completed
